# Collect Basic Data from zaubacorp ([ColabLink](https://colab.research.google.com/drive/1_dd3q87cpQmp9HS-OHIg2tMkI7vIXnq8#scrollTo=t2KigaPJpmKU))

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from tqdm.notebook import tqdm_notebook,tqdm
from selenium.webdriver.common.by import By
from IPython.display import display
# from requestium import Session, Keys
from selenium import webdriver
from bs4 import BeautifulSoup
# import lxml.html as lh
import pandas as pd
import numpy as np
import pytesseract
# import requests
import json
import time
import sys
import re
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
url = 'https://www.zaubacorp.com/user/login?destination=node'
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get(url)

In [ ]:
element_ = driver.page_source
# print(element_)

In [ ]:
def login(cuser_text, pass_text, captcha_text):
    ### Login and access page
    cuser = driver.find_element(By.ID, "edit-name")
    cpass = driver.find_element(By.ID, "edit-pass")
    cpt = driver.find_element(By.ID, "edit-captcha-response")

    cuser.send_keys(cuser_text)
    cpass.send_keys(pass_text)
    cpt.send_keys(captcha_text)

    driver.find_element(By.ID, "edit-submit").click()
    # with codecs.open("/content/drive/MyDrive/Colab Notebooks/IRE Data collection/page_source.txt", "w") as fp:
    #     fp.write(driver.page_source)

In [ ]:
# find Captcha & Solve
t = driver.find_element(By.XPATH,"//div[@class='form-type-textfield form-item-captcha-response form-item form-group']")
# print(t.text)
variables = ''.join(list(filter(lambda x:len(x)!=0,re.split(r'[Math question *]*',t.text)))[:-1])
# print(variables)
ans = eval(variables)
login('prince404','PRINCE@123',str(ans))

In [ ]:
print(driver.current_url)
# this wait below is not working
WebDriverWait(driver=driver, timeout=10).until(
    lambda x: x.execute_script('return document.readyState') == 'complete'
)
print(driver.current_url)
time.sleep(7)
print(driver.current_url)

In [ ]:
def display_all(data):
    for i in range(len(data)):
        print(i)
        display(data[i])

In [ ]:
def get_basic_data(data,d_company):
    temp = np.array(data[0])
    for i in range(len(temp)):
        if((temp[i][1]!='-') and (temp[i][1]!='') and (str(temp[i][1])!='nan')):
            d_company[temp[i][0]]=temp[i][1]

def previous_data(data,d_company,val):
    temp = (np.array(data[0])).tolist()
    if(len(temp)<=1):
        return
    d_company[val] = temp

def establishments(data,d_company):
    temp = np.array(data[0]).tolist()
    val = []
    if(len(temp)<=1):
        return
    for i in range(len(temp)):
        val.append(temp[i][3])
    d_company['Establishents']=val

def charges(data,d_company):
    temp = np.array(data[0]).tolist()
    charges = []
    amount = 0
    if(len(temp)<=1): 
        return
    for i in range(len(temp)):
        if(temp[i][5]!=float('nan') or str(temp[i][5]).isnumeric()):
            amount+=int(temp[i][5])
        charges.append(int(temp[i][0]))
    d_company['Charges IDs'] = charges
    d_company['Cahrges/Borrowing Amount'] = amount
    d_company['Number of Charges']=len(temp)

def persecution(data,d_company):
    temp = np.array(data[0]).tolist()
    # active=0
    # not_active=0
    if(len(temp)<=1):
        return
    d_company['Number of Persecutions']=len(temp)

def cur_directors(data,d_company):
    temp = np.array(data[0]).tolist()
    direct = []
    if(len(temp)<1):
        return
    for i in range(len(temp)):
        if(str(temp[i][0]).isdecimal()):
            direct.append(temp[i][1])
    d_company['Current Directors'] = direct


In [ ]:
def fetch_data(link):
    driver.get(link)
    WebDriverWait(driver=driver, timeout=10).until(
    lambda x: x.execute_script('return document.readyState') == 'complete'
    )
    print(driver.current_url)
    time.sleep(1.5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    temp_data = soup.find_all('table')
    # temp_data = pd.read_html(driver.page_source,header=None)
    d_company = {}
    #NOTE Company Details
    get_basic_data(pd.read_html(str(temp_data[0]),header=None),d_company)

    #NOTE Share Capital & Number of Employees some data Available After Login
    get_basic_data(pd.read_html(str(temp_data[3]),header=None),d_company)

    #NOTE listing status
    get_basic_data(pd.read_html(str(temp_data[4]),header=None),d_company)

    #NOTE Balance Sheet
    get_basic_data(pd.read_html(str(temp_data[5]),header=None),d_company)

    #NOTE Profit & Loss
    get_basic_data(pd.read_html(str(temp_data[6]),header=None),d_company)

    try:
        #NOTE Previous Company CIN available After login
        previous_data(pd.read_html(str(temp_data[2]),header=None),d_company,'Previous CIN')
        #NOTE Previous Company Names available After login
        previous_data(pd.read_html(str(temp_data[1]),header=None),d_company,'Previous Names')
    except:
        pass
    # Establishments :
    try:
        cur_directors(pd.read_html(str(temp_data[7]),header=None),d_company)
        persecution(pd.read_html(str(temp_data[-3]),header=None),d_company)
        charges(pd.read_html(str(temp_data[-2]),header=None),d_company)
        establishments(pd.read_html(str(temp_data[-1]),header=None),d_company)
    except:
        pass

    # print(json.dumps(d_company,indent=5))
    # display_all(temp_data)
    return d_company

In [ ]:
with open('../../data/zaubacorp.com/links.json','r+') as jsonfp:
    company_links = json.load(jsonfp)
    
print("Length of Data :",len(company_links))
company_data = {}
with tqdm_notebook(total=len(company_links)) as pbar:
    for i in company_links:
        json_data = fetch_data(company_links[i]['link'])
        company_data[i] = json_data
        pbar.update()

In [ ]:
file = open("../../data/zaubacorp.com/Companies.json", "w+")
json.dump(company_data, file, indent=4)
file.close()

In [ ]:
# json_file = fetch_data("https://www.zaubacorp.com/company/BHARAT-BIOTECH-INTERNATIONAL-LIMITED/U24230TG1996PLC023232")
# print(json.dumps(json_file,indent=5))